In [ ]:
!pip install flask_cors

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cd /content/drive/MyDrive/mini


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/bin/bash: line 0: cd: /content/drive/MyDrive/mini: No such file or directory


In [ ]:
!pip install pyngrok --quiet
!pip install flask_ngrok
!pip install efficientnet
from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2OYSq0Xys7Imo5NCyKsgapyrMyR_3u41TPKJ3duRUV65qsySg"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


CODE 


In [ ]:
from flask import Flask, request, jsonify, redirect, url_for
from flask_cors import CORS
import os
from werkzeug.utils import secure_filename
from flask_ngrok import run_with_ngrok
import warnings
import cv2 as cv
import numpy as np
from keras.models import load_model
import efficientnet.keras as efn



In [ ]:

# Init app
app = Flask(__name__, template_folder='templates')
CORS(app, resources={r"/*": {"origins": "*"}})
run_with_ngrok(app)


@app.route('/', methods=['GET'])
def get():
    return jsonify({'msg': "health check"})


UPLOAD_FOLDER = './public'
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)
if not os.access(UPLOAD_FOLDER, os.W_OK):
    raise ValueError('Upload folder is not writable')

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER



def find_label_and_predict(image):
    model_folder = './drive/MyDrive/Flask_Server/models/'
    model_fname = 'effnet.h5'

    opencvImage = cv.cvtColor(np.array(image), cv.COLOR_RGB2BGR)
    img = cv.resize(opencvImage, (150, 150))
    img = img.reshape(1, 150, 150, 3)

    model_2 = load_model(os.path.join(model_folder, model_fname))
    p_2 = model_2.predict(img)
    p_2 = np.argmax(p_2, axis=1)[0]

    predictions = {0: 'Glioma Tumor',
                   1: 'No Tumor',
                   2: 'Meningioma Tumor',
                   3: 'Pituitary Tumor'}

    res_2 = predictions[p_2]
    result_str = str(res_2)
    print(result_str)
    return {result_str}


In [ ]:

@app.route('/uploader', methods=['GET', 'POST'])
def upload_files():
    if request.method == 'POST':
        file = request.files['file']

        if file:
            filename = secure_filename(file.filename)
            file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(file_path)
            
            image = cv.imread(file_path)
            res = find_label_and_predict(image)
            print("res="+str(res))
            return str(res)
        else:
            return "No file was uploaded."




In [ ]:
def add_cors_headers(response):
    response.headers['Access-Control-Allow-Origin'] = '*'
    response.headers['Access-Control-Allow-Headers'] = 'Content-Type,Authorization'
    response.headers['Access-Control-Allow-Methods'] = 'GET,PUT,POST,DELETE,OPTIONS'
    return response

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://1e4e-34-147-59-114.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
1/1 [==============================] - 3s 3s/step


INFO:werkzeug:127.0.0.1 - - [18/Apr/2023 12:55:06] "POST /uploader HTTP/1.1" 200 -


Meningioma Tumor
res={'Meningioma Tumor'}
1/1 [==============================] - 2s 2s/step


...

No Tumor
res={'No Tumor'}
1/1 [==============================] - 2s 2s/step


INFO:werkzeug:127.0.0.1 - - [18/Apr/2023 12:55:42] "POST /uploader HTTP/1.1" 200 -


Glioma Tumor
res={'Glioma Tumor'}
1/1 [==============================] - 2s 2s/step


INFO:werkzeug:127.0.0.1 - - [18/Apr/2023 12:56:16] "POST /uploader HTTP/1.1" 200 -


Meningioma Tumor
res={'Meningioma Tumor'}
